In [1]:
!pip install transformers torch pillow tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 101.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 81.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 64.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [2]:
import os
import json
from PIL import Image
from tqdm import tqdm
from transformers import BlipProcessor, BlipForConditionalGeneration

In [3]:
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

In [4]:
!unzip goPro.zip

Archive:  goPro.zip
   creating: goPro/
  inflating: goPro/G0040503.JPG      
  inflating: goPro/G0061996.JPG      
  inflating: goPro/G0083423.JPG      
  inflating: goPro/G0083741.JPG      
  inflating: goPro/G0083742.JPG      
  inflating: goPro/G0083893.JPG      
  inflating: goPro/G0093962.JPG      
  inflating: goPro/G0094034.JPG      
  inflating: goPro/G0145306.JPG      
  inflating: goPro/G0145536.JPG      
  inflating: goPro/G0156860.JPG      
  inflating: goPro/G0167284.JPG      
  inflating: goPro/G0229623.JPG      
  inflating: goPro/G0291345.JPG      
  inflating: goPro/G0291347.JPG      
  inflating: goPro/G0291590.JPG      
  inflating: goPro/G0302024.JPG      
  inflating: goPro/G0322420.JPG      
  inflating: goPro/G0322473.JPG      
  inflating: goPro/G0363233.JPG      
  inflating: goPro/G0363256.JPG      
  inflating: goPro/G0373695.JPG      
  inflating: goPro/G0394107.JPG      
  inflating: goPro/G0394109.JPG      
  inflating: goPro/G0394873.JPG      
  inflatin

In [5]:
IMAGE_FOLDER = "/content/goPro"

In [11]:
OUTPUT_FILE = "/content/image_captions.json"
NUM_CAPTIONS = 3

In [7]:
image_files = [f for f in os.listdir(IMAGE_FOLDER) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

In [8]:
# Container for captions
results = []

In [9]:
# Function to generate N captions using different prompts
def generate_captions(image, n=3):
    prompts = [
        "a photo of",
        "describe this image:",
        "what do you see?"
    ]
    captions = set()
    for i in range(n):
        prompt = prompts[i % len(prompts)]  # rotate prompts if n > len(prompts)
        inputs = processor(image, prompt, return_tensors="pt")
        out = model.generate(**inputs, max_length=50, num_beams=5)
        caption = processor.decode(out[0], skip_special_tokens=True)
        captions.add(caption)
    return list(captions)


In [12]:
for filename in tqdm(image_files, desc="Captioning images"):
    image_path = os.path.join(IMAGE_FOLDER, filename)
    image = Image.open(image_path).convert("RGB")
    captions = generate_captions(image, NUM_CAPTIONS)
    results.append({
        "image": filename,
        "captions": captions
    })

Captioning images: 100%|██████████| 95/95 [58:14<00:00, 36.78s/it]


In [13]:
with open(OUTPUT_FILE, "w") as f:
    json.dump(results, f, indent=2)